In [ ]:
# clean the memory
%reset
!pip install pycuda

In [ ]:
import numpy as np
from pycuda import gpuarray, autoinit
import pycuda.driver as cuda
from pycuda.tools import DeviceData
from pycuda.tools import OccupancyRecord as occupancy

In [ ]:
presCPU, presGPU = np.float32, 'float'
#presCPU, presGPU = np.float64, 'double'
a_cpu = np.random.random((512,512)).astype(presCPU)
b_cpu = np.random.random((512,512)).astype(presCPU)
c_cpu = np.zeros((512,512), dtype=presCPU)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(a_cpu)
plt.colorbar()

In [ ]:
plt.imshow(b_cpu)
plt.colorbar()

In [ ]:
# Array on GPU
a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)
c_gpu = gpuarray.to_gpu(c_cpu)

In [ ]:
c_gpu

In [ ]:
c_cpu=a_cpu+b_cpu

In [ ]:
c_cpu

In [ ]:
t_cpu = %timeit -o c_cpu = a_cpu+b_cpu

In [ ]:
# kernel gpu - sum

cudaKernel = '''
__global__ void matrixAdd(float *A, float *B, float *C)
{
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
    int tid   = gridDim.x * blockDim.x * tid_y + tid_x;
    C[tid] = A[tid] + B[tid];
}
'''

In [ ]:
# compile the kernel and generate the function to use in python

from pycuda.compiler import SourceModule
myCode = SourceModule(cudaKernel)

In [ ]:
addMatrix = myCode.get_function("matrixAdd") # The output of get_function is the GPU-compiled function.

In [ ]:
type(addMatrix)

In [ ]:
# GPU geometry of interest. We can use all the threads in a block. How many threads are in a block?

dev = cuda.Device(0)
devdata = DeviceData(dev)
print ("Using device : "+dev.name() )
print("Max threads per block: "+str(dev.max_threads_per_multiprocessor))

In [ ]:
#so we can use 32x32 blocks. Our matrices are 512x512, so we need to use 16x16 blocks

cuBlock = (32,32,1)
cuGrid = (16,16,1)

after the first compilation, the kernel is already compiled and we can call it directly with the function name. 

```
kernelFunction(arg1,arg2, ... ,block=(n,m,l),grid=(r,s,t))
```

We can also use the "preparation" method, which is more efficient when we have to call the kernel many times.

```
kernelFunction.prepare('ABC..') # Each letter corresponds to an input data type of the function, i = int, f = float, P = pointer, ...
kernelFunction.prepared_call(grid,block,arg1.gpudata,arg2,...) # When using GPU arrays, they should be passed as pointers with the attribute 'gpudata'
```

In [ ]:
# first method
addMatrix(a_gpu,b_gpu,c_gpu,block=cuBlock,grid=cuGrid)

In [ ]:
# second method
addMatrix.prepare('PPP')
addMatrix.prepared_call(cuGrid,cuBlock,a_gpu.gpudata,b_gpu.gpudata,c_gpu.gpudata)

In [ ]:
time2 = addMatrix.prepared_timed_call(cuGrid,cuBlock,a_gpu.gpudata,b_gpu.gpudata,c_gpu.gpudata)

In [ ]:
time2()

In [ ]:
# copy the result from GPU to CPU
c = c_gpu.get()

In [ ]:
c, c_cpu

In [ ]:
plt.imshow(c-c_cpu,interpolation='none')
plt.colorbar()

In [ ]:
np.sum(np.sum(np.abs(c_cpu-c)))